__Toutes les fonctions définies dans le notebook pre_processing sont mises dans un fichier Python qu'on importe__

In [1]:
import os
import dicom
import pylab
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
from skimage.filters import roberts, sobel
from skimage.segmentation import clear_border
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing

import pre_processing as pp

In [2]:
INPUT_FOLDER = '/home/lucsoret/Projet/Supelec/CEI/Data/sample_images'
patients = os.listdir(INPUT_FOLDER)
patients.sort()

__Workflow__

In [6]:
first_patient_path = os.path.join(INPUT_FOLDER,patients[0])
first_patient = pp.load_scan(first_patient_path)
total_images = os.listdir(first_patient_path)


first_patient_image =  pp.PixelData_to_HU(first_patient)
first_patient_image =  np.stack([pp.center(pp.normalize(pp.get_segmented_lungs(s))) for s in image])